In [ ]:
!nvidia-smi

Fri Mar 19 16:50:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install torch
!pip install tweet-preprocessor
!pip install bs4
!pip install sentencepiece 
!pip install langdetect
!pip install translate-api

     |████████████████████████████████| 2.0MB 8.4MB/s 
     |████████████████████████████████| 3.2MB 50.1MB/s 
     |████████████████████████████████| 890kB 54.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=685bcd640a8eae6121f47cc4daf191cd10be7413c3191a1c30f0bb1803b27b33
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 9.9MB/s 
     |████████████████████████████████| 983kB 7.1MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=a1bc019b96491934fb9f1769fd030a1f0d6416e2c8e59f46320352b489efa99a
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 1.0MB 10.5MB/s 
     |████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd
import preprocessor as p
from bs4 import BeautifulSoup
import re
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import class_weight

# Data Import

In [ ]:
db1=pd.read_excel("/content/drive/MyDrive/H2_B21_14/Development Data/dev_data_article.xlsx")
db2=pd.read_excel("/content/drive/MyDrive/H2_B21_14/Development Data/dev_data_tweet.xlsx")
db1=db1.drop_duplicates(subset=['Text'],ignore_index=True)    #Removing duplicates
db2=db2.drop_duplicates(subset=['Tweet'],ignore_index=True)  

In [ ]:
feature_art=list(db1['Text'])
label_art=list(db1['Mobile_Tech_Flag'])

In [ ]:
counter_art=0
for x in label_art:
  if(x==1):
    counter_art+=1

In [ ]:
print(f"Number of mobile centric articles out of {len(label_art)} is {counter_art}")

Number of mobile centric articles out of 3567 is 873


In [ ]:
feature_tweet=list(db2['Tweet'])
label_tweet=list(db2['Mobile_Tech_Tag'])

In [ ]:
counter_tweet=0
for x in label_tweet:
  if(x==1):
    counter_tweet+=1

In [ ]:
print(f"Number of mobile centric tweets out of {len(label_tweet)} is {counter_tweet}")

Number of mobile centric tweets out of 2400 is 569


# Preprocessing

Preprocessing Tweets

In [ ]:
#Basic Preprocessing
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
feature_tweet1=[]
for x in range(len(feature_tweet)):
  m=p.clean(feature_tweet[x])
  m=m[len('QT '):] if m.startswith('QT ') else m
  for y in punc:
    m=m[len(y):] if m.startswith(y) else m
  m.strip()
  feature_tweet1.append(m)

feature_tweet1=[re.sub('\n+','',t) for t in feature_tweet1]
#Removing null values
assert len(feature_tweet)==len(feature_tweet1)
ft1=[]
lb1=[]

for x in range(len(feature_tweet1)):
  if(len(feature_tweet1[x])>5):
    ft1.append(feature_tweet1[x])
    lb1.append(label_tweet[x])

feature_tweet_pp=ft1
label_tweet_pp=lb1


Preprocessing Articles

In [ ]:
def remove_html(word):
  soup = BeautifulSoup(word, 'lxml')
  html_free = soup.get_text()
  return html_free

def remove_urls(word):
    url_pattern = re.compile(r'https?:\/\/.*[\r\n]*')
    return url_pattern.sub(r'', word)

ft2=[]
lb2=[]
feature_art_px=[]
for x in range(len(feature_art)):
  temp=remove_html(feature_art[x])
  temp=remove_urls(temp)
  idx=temp.find('\n')
  subw=temp[:idx]
  if(len(subw)<50 or len(subw.split(" "))<7):
    feature_art_px.append(temp[idx:])
  else:
    feature_art_px.append(temp)

feature_art_px=[re.sub('\n+','',t) for t in feature_art_px]

for x in range(len(feature_art_px)):
  if(len(feature_art_px[x])>5):
    ft2.append(feature_art_px[x])
    lb2.append(label_art[x])

feature_art_pp=ft2
label_art_pp=lb2


In [ ]:
len(feature_tweet_pp)


2347

Translation

In [ ]:
import translators as ts

In [ ]:
feature_art_final=[]
feature_tweet_final=[]
count=0;

for a in feature_art_pp:
  if(len(a)>5000):
    a=a[:4999]
  if(a!=''):
    feature_art_final.append(ts.google(str(a))) 
    time.sleep(0.5)
  else:
    feature_art_final.append('')
  print(f"Example number {count}")
  count+=1

count=0

for a in feature_tweet_pp:
  if(len(a)>5000):
    a=a[:4999]
  if(a!=''):
    feature_tweet_final.append(ts.google(str(a))) 
    time.sleep(0.5)
  else:
    feature_tweet_final.append('')
  print(f"Example number {count}")
  count+=1

In [ ]:
### Saving dataframe post translation:
df=pd.DataFrame(list(zip(list(feature_art_final)+list(feature_tweet_final),list(label_art_pp)+list(label_tweet_pp))),columns=['Features','Labels'])
df.to_csv("/content/drive/MyDrive/H2_B21_14/Preprocessed_Dataset.csv")

#Training

### Distilbert

In [ ]:
dff=pd.read_csv("/content/drive/MyDrive/H2_B21_14/Preprocessed_Dataset.csv")

In [ ]:
net_features=dff['Features']
net_labels=dff['Labels']

In [ ]:
import random
temp = list(zip(net_features, net_labels)) 
random.shuffle(temp) 
net_features, net_labels = zip(*temp)

In [ ]:
training_size=round(len(net_features)*0.8)

In [ ]:
len(net_features)
assert len(net_features)==len(net_labels)

In [ ]:
training_sentences = list(net_features[0:training_size])
validation_sentences = list(net_features[training_size:])
training_labels = list(net_labels[0:training_size])
validation_labels = list(net_labels[training_size:])

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))


In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
weights = class_weight.compute_class_weight('balanced',
                                            np.unique(training_labels),
                                            training_labels)

In [ ]:
a={0:weights[0],1:weights[1]}

In [ ]:
callback=tf.keras.callbacks.EarlyStopping(patience=2)

In [ ]:
  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
  h=model.fit(train_dataset.shuffle(100).batch(16),
          epochs=20,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16),
          class_weight=a,callbacks=[callback])

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autog

In [ ]:
model.save_pretrained("/content/drive/MyDrive/H2_B21_14/DistilBERT")

Prediction

In [ ]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/H2_B21_14/DistilBERT")

Some layers from the model checkpoint at /content/drive/MyDrive/Inter IIT - Bridgei2i/Sentiment Classification Model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Inter IIT - Bridgei2i/Sentiment Classification Model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be ab

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/H2_B21_14/Preprocessed_Dataset.csv")

In [ ]:
feat=df['Features']
label=df['Labels']

In [ ]:
predicted_labels=[]
counter=0
for y in feat:
  predict_input = tokenizer.encode(y,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
  tf_output = loaded_model.predict(predict_input)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
  predicted_labels.append(int(tf.argmax(tf_prediction)))
  print(f"Example {counter}")
  counter+=1

In [ ]:
m1=tf.keras.metrics.Recall()
m2=tf.keras.metrics.Precision()
m1.update_state(label,predicted_labels)
m2.update_state(label,predicted_labels)

In [ ]:
recall=m1.result().numpy()
precision=m2.result().numpy()
print(f"Recall = {recall}")
print(f"Precision = {precision}")
f1=2*recall*precision/(recall+precision)
print(f"F1 = {f1}")

Recall = 0.9941002726554871
Precision = 0.9519773721694946
F1 = 0.9725829765706907


In [ ]:
df['Predicted']=predicted_labels

In [ ]:
df.to_csv("/content/drive/MyDrive/H2_B21_14/Preprocessed_Dataset.csv")

# Entity Level Sentiment Analysis

In [ ]:
df=df[df.Labels==1]

In [ ]:
df

In [ ]:
!pip install aspect-based-sentiment-analysis

In [ ]:
import aspect_based_sentiment_analysis as absa


nlp = absa.load()


In [ ]:
mobile_companies = ['acer','amazon','apple','asus','at&t','blackberry','bosch','casio','celkon','chea','dell','ericsson','gigabyte','gionee','google','haier','honor','hp','htc','huawei','i-mate','infinix','intex','karbonn','lava','leeco','lenovo','lg','micromax','microsoft','motorola','nokia','nvidia','oneplus','oppo','panasonic','philips','realme','samsung','sony','sony-ericsson','spice','toshiba','vivo','vodafone','xiaomi','moto','redmi','iqoo']

In [ ]:
trt=['Deleted Photo (Images) Ko Recover Kaise Kare Mobile Phone Me	Recover Deleted Photo',
     'Android Mobile Me Paise Kamaye mCent App Install karke	Earn money through mCent App Android Mobile',
     'Mobile Me App Install karke Paise Kaise Kamaye [Top 6 App]	Earn money through apps in your Mobile',
     '10 Security Mistakes ?? ?? ???? ?????? ?????? ?? ???? ??	10 Security Mistakes by small businesses',
     'Blogging Tools ?? ???? ??????? ????? Blogger	8 Blogging Tools to be a better Blogger',
     'Redmi 9 Power ?????????? ?? 6GB RAM ??????? ???? ??? ?????	Redmi 9 Power smartphone\'s 6GB RAM variant launched in India',
     '?? Digital marketing kya hai? or Isme expert kaise bane?	What is Digital marketing and how to be expert',
     'Great deal to buy Xiaomi 5G smartphone'
     'iQOO brings 120W 15 minute fast charge smartphone to India',
     'Incredible Samsung Galaxy F62 7000mah battery smartphone Sale Tomorrow',
     'Xiaomi\'s new variant of 6000mAh battery 48MP camera will launch soon',
     'Nokia 3.4 or Moto E7 Power. Similar features but price differs',
     'Nokia\'s 4GB RAM  4000mAh battery and 3 camera smartphone\'s sale starts tomorrow',
     'WhatsApp assures the Indian government to the protection of privacy of personal conversations',
     'Oppo FindX3 will not be powered by Qualcomm\'s flagship processor']

In [ ]:
brand_found=[]
sentiment=[]
for i,x in enumerate(trt):
  temp=[]
  temp1=[]
  x=' '+x+' '
  x = re.sub('([.,!?()])', r' \1 ', x) 
  x = re.sub('\s{2,}', ' ', x)
  for y in mobile_companies:
    
    if(' '+y+' ' in x.lower() or ' '+y in x.lower()):
      idx=x.lower().find(y)
      x0=str(x[max(idx-250,0):min(idx+250,len(x))]).lower()
      temp.append(y)
      sent=nlp((x0),aspects=[y])
      if(int(sent.subtasks[y].examples[0].sentiment)==2):
        temp1.append('postive')
      elif(int(sent.subtasks[y].examples[0].sentiment)==1):
        temp1.append('negative')
      else:
        temp1.append('neutral')
    
  brand_found.append(temp)
  sentiment.append(temp1)
  print(i)

In [ ]:
brand_found

In [ ]:
sentiment